In [114]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
pd.options.mode.chained_assignment = None
import random

from recoxplainer.config import cfg
from recoxplainer.data_reader import DataReader 
from recoxplainer.models import ALS, EMFModel, GMFModel,BPR, PyTorchModel, ExplAutoencoderTorch
from recoxplainer.models.mlp_model import MLPModel
from recoxplainer.recommender import Recommender 
from recoxplainer.evaluator import Splitter, Evaluator
from recoxplainer.explain import ALSExplainer
from recoxplainer.evaluator import ExplanationEvaluator

In [115]:
data = DataReader(**cfg.ml100k)

In [116]:
data.make_consecutive_ids_in_dataset()

In [117]:
data.binarize(binary_threshold=1)

In [118]:
sp = Splitter()
train, test = sp.split_leave_n_out(data, frac=0.1)

# Testing ALS

In [121]:
als = ALS(latent_dim=16, epochs=30 , reg_term= 0.005)

In [122]:
print(als.reg_term)

0.005


In [123]:
als.fit(train)

/Users/mac/opt/anaconda3/envs/recoxplainer/lib/python3.6/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



  0%|          | 0/30 [00:00<?, ?it/s]

True

In [124]:
recommender = Recommender(train, als)

In [125]:
recommendations = recommender.recommend_all()

Recommending for users:   0%|          | 0/943 [00:00<?, ?it/s]

In [126]:
eva = Evaluator(test)

In [127]:
eva.cal_hit_ratio(recommendations)

0.26530880160637865

In [128]:
eva.cal_ndcg(recommendations)

0.32809923658336826

# Testing EMF

In [13]:
emf = EMFModel(**cfg.model.emf)

In [14]:
emf.fit(train)

  0%|          | 0/10 [00:00<?, ?it/s]

True

In [17]:
recEMF = Recommender(train, emf)
recEMF = recEMF.recommend_all()

Recommending for users:   0%|          | 0/943 [00:00<?, ?it/s]

In [18]:
eva.cal_hit_ratio(recEMF)

0.030749810797047666

In [20]:
eva.cal_ndcg(recEMF)

0.039184881904460934

# Testing GMF

In [92]:
gmf = GMFModel(**cfg.model.gmf)
gmf.fit(train)

  0%|          | 0/30 [00:00<?, ?it/s]

True

In [93]:
recGMF = Recommender(train, gmf)
RecsGMF = recGMF.recommend_all()

Recommending for users:   0%|          | 0/943 [00:00<?, ?it/s]

In [94]:
eva.cal_hit_ratio(RecsGMF)

0.0005802992812536821

In [95]:
eva.cal_ndcg(RecsGMF)

0.0009633867485160594

# Testing BPR

In [129]:
bpr = BPR(latent_dim=16, epochs=100 , reg_term= 0.0, learning_rate=0.005)
bpr.fit(train)

/Users/mac/opt/anaconda3/envs/recoxplainer/lib/python3.6/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



  0%|          | 0/100 [00:00<?, ?it/s]

True

In [134]:
bpr.epochs

100

In [130]:
recBPR = Recommender(train, bpr)
RecsBPR = recBPR.recommend_all()

Recommending for users:   0%|          | 0/943 [00:00<?, ?it/s]

In [131]:
eva.cal_hit_ratio(RecsBPR)

0.2232330454840628

In [132]:
eva.cal_ndcg(RecsBPR)

0.2664102389638914

# Testing MLP

In [59]:
mlp = MLPModel(**cfg.model.mlp)
mlp.fit(train)

  0%|          | 0/30 [00:00<?, ?it/s]

True

In [76]:
mlp.learning_rate

0.005

In [60]:
recMLP = Recommender(train, mlp)
RecsMLP = recMLP.recommend_all()

Recommending for users:   0%|          | 0/943 [00:00<?, ?it/s]

In [61]:
eva.cal_hit_ratio(RecsMLP)

0.0062603283049068505

In [62]:
eva.cal_ndcg(RecsMLP)

0.007851105857529975